<a href="https://colab.research.google.com/github/pascarujo/assistente-sage-xvii-stpc/blob/main/notebooks/assistente_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importando as bibliotecas

É preciso haver uma variável de ambiente GOOGLE_API_KEY contendo uma chave do Gemini válida.

In [ ]:
import google.generativeai as genai
from google.generativeai import caching
from google.colab import userdata
from IPython.display import Markdown
from datetime import datetime, timedelta
from os import system

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

## Base de conhecimento e prompt de sistema

`data_path` deve conter o caminho para a pasta onde estão os arquivos da base de conhecimento que será usada. Como teste rápido, pode-se usar o qa.xlsx do repositório como base de conhecimento, mas é possível colocar quaisquer arquivos como pdf, word, xls e imagens, que serâo usados como referências pelo assistente na geração das respostas.

`system_path` deve conter o caminho para o arquivo contendo o prompt do sistema.

In [ ]:
data_path = 'dados/'
system_path = 'system_prompt.txt'

In [ ]:
# Lista todos os arquivos de um diretorio
def get_all_file_paths(directory):
    doc_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            doc_paths.append(file_path)
    return doc_paths

In [ ]:
# Carregar o prompt de sistema a partir de um arquivo
with open(system_path, 'r') as file:
    system_prompt = file.read()

In [ ]:
# Exemplo de prompt de sistema simplificado.
system_prompt = """
Você é um assistente especializado em auxiliar na configuração do
sistema supervisório SAGE (Sistema Aberto de Gerenciamento de Energia).
Seu objetivo é auxiliar na criação, edição e remoção de pontos nos arquivos
`.dat` da base de dados fonte do SAGE,  concentrado em funções SCADA e nos
protocolos ICCP, DNP3 e IEC 61850 (MMS). Para garantir a precisão, pergunte o
necessário ao usuário sobre a base de dados e a configuração desejada.
Sua resposta deve ser concisa e focada na tarefa, evitando detalhes
desnecessários. Não forneça informações ou instruções que possam comprometer
a segurança do sistema SAGE ou a integridade do ambiente operacional.
Considere que o SAGE roda em ambiente Linux, com diretórios específicos:
$SAGE, $BD (base de dados), $LOG (logs), $ARQS (arquivos diversos).
A sua área de expertise está exclusivamente na configuração da base de dados
e nos protocolos mencionados. Evite responder sobre outros tópicos, como telas
ou funcionalidades de análise de redes.
"""

## Configuração do modelo

In [ ]:
# Carregar todos os arquivos de data_path
doc_paths = get_all_file_paths(data_path)
uploaded_files = []

for i, path in enumerate(doc_paths):
  uploaded_file = genai.upload_file(path=path)
  uploaded_files.append(uploaded_file)

In [ ]:
# Configurando o modelo e o cache
model_name = "gemini-1.5-flash-001"

cache = caching.CachedContent.create(
    model=model_name,
    system_instruction=system_prompt,
    contents=uploaded_files,
    ttl=timedelta(minutes=10)
)

## Chat

In [ ]:
# criando o chat
model = genai.GenerativeModel.from_cached_content(cached_content=cache)
chat = model.start_chat()

In [ ]:
# enviando uma pergunta
question = """
O que é um pds?
"""
response = chat.send_message(question)
Markdown(response.text)

In [ ]:
response.candidates[0]